# Forged Dataset Generation(in same image)


In [ ]:
base_dir = "/content/data"

# Create the base directory
os.makedirs(base_dir, exist_ok=True)

# Define class names
class_names = ["Forged", "Mask"]

# Loop through the class names and create subdirectories
for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

In [ ]:

def gen(field_coords,data,last,index,img, image_name):
    length = len(field_coords)
    for i in range(length):

        data[index] = field_coords[i]  # Store the field and coordinates

        if index == last:  # If we have a complete permutation
            new_img = img.copy()
            forged_mask = np.zeros_like(img[:, :, 0])

            forged_name = f"{image_name[:-4]}_{'_'.join([f'{field_name[0]}' for i, field_name in enumerate(data)])}.jpg"


            for i, field_name in enumerate(data):

              for j, (other_field_name, other_coords) in enumerate(field_coords):
                if i == j:
                  dest_coords = other_coords

              for j,(other_field_name, other_coords) in enumerate(field_coords):

                if (field_name[0]==other_field_name and j!=i):
                  forged_mask[dest_coords[0]['y']:dest_coords[0]['y'] + dest_coords[0]['height'],
                            dest_coords[0]['x']:dest_coords[0]['x'] + dest_coords[0]['width']] = 255

                  forged_mask_3_channel = cv2.cvtColor(forged_mask, cv2.COLOR_GRAY2BGR)  # Convert to BGR if needed
                  mask_image_save_path = os.path.join(base_dir, "Mask", forged_name)
                  cv2.imwrite(mask_image_save_path, forged_mask_3_channel)


            for i, field_name in enumerate(data):

              for j,(other_field_name, other_coords) in enumerate(field_coords):
                if i==j:
                  dest_coords=other_coords

              for j,(other_field_name, other_coords) in enumerate(field_coords):

                if (field_name[0]==other_field_name and j!=i):


                  src_region = img[other_coords[0]['y']:other_coords[0]['y'] + other_coords[0]['height'],other_coords[0]['x']:other_coords[0]['x'] + other_coords[0]['width']]

                  src_region_resized = cv2.resize(src_region,
                                                (dest_coords[0]['width'], dest_coords[0]['height']))
                  new_img[dest_coords[0]['y']:dest_coords[0]['y'] + dest_coords[0]['height'],
                  dest_coords[0]['x']:dest_coords[0]['x'] + dest_coords[0]['width']] = src_region_resized
                  forged_image_save_path = os.path.join(base_dir, "Forged", forged_name)

                  cv2.imwrite(forged_image_save_path, new_img)
                  # Upload to GitHub

                  break

                else:
                  continue


        else:
            gen(field_coords, data, last, index + 1, img, image_name)



def generate():
    for i in range(100):
        if(i<10):
            image=f'0{i}.jpg'

        else:
            image=f'{i}.jpg'


        image_Path = repository_path + image
        img = cv2.imread(image_Path)


        with open(json_path) as f:
          data = json.load(f);

        # Extract metadata paths
        region = [data['_via_img_metadata'][key]['regions']
                  for key in data['_via_img_metadata']]
        reg = region[i]

        birth_date_coords = []
        expiry_date_coords = []
        issue_date_coords = []



        surname_coords = []
        name_coords = []



        nationality_coords = []
        birth_place_coords = []


        number_coords = []
        id_number_coords = []



        for region in reg:
          if region['region_attributes']['field_name'] == 'birth_date':
            birth_date_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'expiry_date':
            expiry_date_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'issue_date':
            issue_date_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'surname':
            surname_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'name':
            name_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'nationality':
            nationality_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'birth_place':
            birth_place_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'number':
            number_coords.append(region['shape_attributes'])
          elif region['region_attributes']['field_name'] == 'id_number':
            id_number_coords.append(region['shape_attributes'])
        group1=[('birth_date', birth_date_coords), ('expiry_date', expiry_date_coords), ('issue_date', issue_date_coords)]
        group2=[('surname', surname_coords), ('name', name_coords)]
        group3=[('nationality', nationality_coords), ('birth_place', birth_place_coords)]
        group4=[('number', number_coords), ('id_number', id_number_coords)]
        groups = [
            group1,
            group2,
            group3,
            group4
        ]

        # Iterate through each group
        # Iterate through each group
        for group in groups:
            if len(group) < 2:  # Skip groups with less than 2 fields
                continue

            # Get field names from the group
            field_names = [item[0] for item in group]
            all_coords = [item[1] for item in group]
            length=len(group)
            data=[None] * length

            gen(group, data, length-1,0,img,image)


generate()